In [1]:
import os
os.chdir(".././")

In [2]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.document_loaders import TextLoader

In [3]:
#load pdf files
def load_pdf_files(path: str) -> list[str]:
    loader=DirectoryLoader(path,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents

In [4]:
extracted_data = load_pdf_files(path="data/pdf/")

In [5]:
extracted_data

[Document(metadata={'producer': 'LibreOffice 24.8.4.2 (X86_64) / LibreOffice Community', 'creator': 'Calc', 'creationdate': '2025-03-16T09:54:28+00:00', 'source': 'data\\pdf\\data_set.pdf', 'total_pages': 1019, 'page': 0, 'page_label': '1'}, page_content='data_set\nPage 1\nTitle\nCÁC NGÀNH ĐÀO TẠO CỦA ĐẠI HỌC ĐÔNG Á\ncác tổ hợp xét tuyển gồm những môn nào\ncác ngành đào tạo\nmã ngành Y khoa\nmã ngành Dược\nmã ngành Điều dưỡng\nmã ngành Điều dưỡng\nmã ngành Hộ sinh\nmã ngành  Dinh dưỡng\nmã ngành  Kỹ thuật phục hồi chức năng\nmã ngành Tâm lý học\nmã ngành Quản lý văn hóa\nmã ngành Công nghệ thông tin\nmã ngành Công nghệ thông tin tại daklak'),
 Document(metadata={'producer': 'LibreOffice 24.8.4.2 (X86_64) / LibreOffice Community', 'creator': 'Calc', 'creationdate': '2025-03-16T09:54:28+00:00', 'source': 'data\\pdf\\data_set.pdf', 'total_pages': 1019, 'page': 1, 'page_label': '2'}, page_content='data_set\nPage 2\nmã ngành Kỹ thuật máy tính\nmã ngành Trí tuệ nhân tạo\nmã ngành Trí tuệ nhâ

In [6]:
def load_csv_files(path: str) -> list[str]:
    loader=TextLoader(path)
    documents=loader.load()
    return documents

In [7]:
#Split text into chunks
def text_split(extracted_data: str) -> list:
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of text chunks:",len(text_chunks))

Length of text chunks: 3017


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
def hugging_face_embeddings():
   embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
   return embeddings

In [11]:
embeddings = hugging_face_embeddings()

C:\Users\Namca\AppData\Local\Temp\ipykernel_38556\3424792433.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Namca\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query=embeddings.embed_query("hello")
print(len(query))


384


In [13]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [14]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY=os.environ.get('GEMINI_API_KEY')

In [15]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

print(PINECONE_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="test"


index = pc.Index("test")

pcsk_5DkPsy_5zMJfJxYk7ukNxEmtFJGSXhEjSLhdmyjP4AUFo4t7eig3yRijVPqpYwzCsjzuun


In [ ]:
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings
# )

In [17]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)

In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [20]:
retriever_docs = retriever.invoke("Học phí nghành công nghệ thông tin")

In [21]:
retriever_docs

[Document(id='8b75bb0a-aa46-496a-9492-d68bf8e37e01', metadata={'creationdate': '2025-03-16T09:54:28+00:00', 'creator': 'Calc', 'page': 223.0, 'page_label': '224', 'producer': 'LibreOffice 24.8.4.2 (X86_64) / LibreOffice Community', 'source': 'data\\pdf\\data_set.pdf', 'total_pages': 1019.0}, page_content='Học phí bậc đại học chính quy khóa 2024 - Đại học Đông Á xem tại: https://donga.edu.vn/tuyensinh/ts-chi-tiet/hoc-phi-bac-dai-hoc-chinh-quy-khoa-2024---dai-hoc-dong-a-35972\nBạn muốn biết thêm thông tin cụ thể gì về thông tin học phí không? Ví dụ như:\n- Chương trình đào tạo ở khối ngành kỹ thuật và dược, y khoa\n- học phí Chương trình cử nhân\n- học phí NGOẠI NGỮ'),
 Document(id='a972ef0c-2413-4a6e-baf9-00c9e414c872', metadata={'creationdate': '2025-03-16T09:54:28+00:00', 'creator': 'Calc', 'page': 187.0, 'page_label': '188', 'producer': 'LibreOffice 24.8.4.2 (X86_64) / LibreOffice Community', 'source': 'data\\pdf\\data_set.pdf', 'total_pages': 1019.0}, page_content='Việc ứng dụng Côn

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    treaming=True,
    temperature=0,
    api_key=GEMINI_API_KEY,
)

In [23]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = ("""Bạn là trợ lý tư vấn tuyển sinh của trường Đại học Đông Á. Hãy trả lời câu hỏi của học sinh dựa trên thông tin được cung cấp một cách chi tiết và thân thiện.

Nếu không có đủ thông tin để trả lời câu hỏi cụ thể, hãy:
1. Trả lời những phần thông tin mà bạn có
2. Cung cấp đường link chính thức để học sinh tham khảo thêm
3. Gợi ý học sinh có thể hỏi thêm những thông tin liên quan

{context}""")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{{question}}"),
    ]
)


In [24]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [27]:
response = rag_chain.invoke({"input": "Ngành nào đang hot?"})
print(response)

{'input': 'Ngành nào đang hot?', 'context': [Document(id='ac7341b8-a19b-415c-a323-9160023da9ba', metadata={'creationdate': '2025-03-16T09:54:28+00:00', 'creator': 'Calc', 'page': 188.0, 'page_label': '189', 'producer': 'LibreOffice 24.8.4.2 (X86_64) / LibreOffice Community', 'source': 'data\\pdf\\data_set.pdf', 'total_pages': 1019.0}, page_content='Vì là ngành hot nên Công nghệ thông tin được nhiều bạn học sinh ưa chuộng. Tuy nhiên, để tìm một ngôi trường phù hợp, phát triển bản thân thì không hề đơn giản.'), Document(id='af6221ed-e175-4b5e-a0ff-b18b51cd194d', metadata={'creationdate': '2025-03-16T09:54:28+00:00', 'creator': 'Calc', 'page': 285.0, 'page_label': '286', 'producer': 'LibreOffice 24.8.4.2 (X86_64) / LibreOffice Community', 'source': 'data\\pdf\\data_set.pdf', 'total_pages': 1019.0}, page_content='như đánh giá chất lượng của chúng.'), Document(id='c2591193-0df9-46dd-a3e7-d16b84bc6d0f', metadata={'creationdate': '2025-03-16T09:54:28+00:00', 'creator': 'Calc', 'page': 176.0, 